# 04 LSTM Eğitimi

In [1]:
import pandas as pd
import sys
sys.path.append("..")
from src.config import PROJECT_ROOT
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader, random_split


In [2]:
df = pd.read_parquet(PROJECT_ROOT.joinpath('data/interim/sample.parquet'))
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1

data_array = df.values


In [3]:
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])
train_loader = DataLoader(train_ds, batch_size=10, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=10)


In [5]:
sensors = data_array.shape[1]
print(sensors)

8600


In [6]:
predictor = TrafficPredictor('lstm', sensors, X_STEP, Y_STEP)
predictor.train(train_loader, val_loader, epochs=50, 
                save_path=PROJECT_ROOT.joinpath("models/lstm.pt"), 
                save_optimizer=True)
predictor.show()


  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 1/50  Train Loss: 15509.8701  Val Loss: 15244.4375  Val MSE: 15244.4375  Val R2: -1.6511
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 2/50  Train Loss: 15508.5420  Val Loss: 15243.1514  Val MSE: 15243.1514  Val R2: -1.6509
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 3/50  Train Loss: 15507.1953  Val Loss: 15241.7285  Val MSE: 15241.7285  Val R2: -1.6507
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 4/50  Train Loss: 15505.6943  Val Loss: 15240.1250  Val MSE: 15240.1260  Val R2: -1.6504
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt (val_loss improved)
Epoch 5/50  Train Loss: 15504.0098  Val Loss: 15238.3086  Val MSE: 15238.3076  Val R2: -1.6501
  >> Model saved to /home/orhankocak_0233/MLProject_570/models/lstm.pt